### Ce Projet consiste à lancer des **requêtes SQL** sur des données CRM (pipeline de vente B2B) d'une entreprise qui vend du matériel informatique. L'objectif est de monter la capacité à requêter les bases de données SQL pour des besoins métiers: des requêtes les plus simples au plus avancées.

**Axe I: Analyse des opportunités créées**
- `Requête 1` : Selection des opportunités ayant le statut "engagés ou gagnés"
- `Requête 2` : Afficher les opportunités les plus anciennes antérieures à juin 2016 et qui qui sont toujours encours(avec un account renseigné)
- `Requête 3` : Est ce que ce sont les opportunités qui impliquent des produits plus chers vs les opportunités plus récentes?
- `Requête 4` : Selection pour chaque produit du % d'opportunités perdues par rapport au total des opportunités de ce produit

**Axe II: Analyse Produit, Prix et leur évolution dans le temps**
- `Requête 5` : Selection des produits de la séries GTX/MG du plus cher au moins cher (ici séries avec 3/2 caractères)"
- `Requête 6` : Classement automatique des produits en fonction du CA (fonction de Fenetrage)
- `Requête 7` : Quels sont les produits dont on a le plus baissé le prix pour assurer la vente?
- `Requête 8` : Les produits les plus chers ont-ils une durée de négociation plus longue?
- `Requête 9` : Somme des ventes par année (engage_date) / par mois (close_date) et ensuite pour chaque produit 
- `Requête 10`: Calcul du taux d'évolution des ventes par année et par mois

**Axe III: Analyse Entreprise BtoB**
- `Requête 11` : Top 10 des entreprises qui ont en moyenne de jour de négociation plus longue"
- `Requête 12` : Classidfication des clients par catégorie (Grands_comptes/Comptes_moyens/Petits_comptes) en fonction du montant de ventes

**Axe IV: Analyse Performances des employés**
- `Requête 13` : Selection des lignes de la table_vente ayant le statut "engagés ou gagnés"
- `Requête 14` : LeTop 3 des meilleurs employé en terme de CA par année (DENSE_RANK)+ Pour les exaequo sans saut: 

*PS: La majorité de ces questions peuvent être traitées directement avec des codes Python ou des outils de BI mais ici l'exercice consiste à visualiser les requêtes comme si on travaillait avec une base SQL.*


### Import et création de base de données SQL
**Tout d'abord, importons les packages nécéssaires et les fichiers csv pour reproduire nos tables SQL**

In [2]:
#import packages
import sqlite3 as sql
import pandas as pd
#import fichier
path=./SQL"
ventes=pd.read_csv(Ventes.csv, sep=";")
produit=pd.read_csv(Produits.csv, sep=";")
Employés=pd.read_csv(Employés.csv, sep=";")
clients=pd.read_csv(Clients.csv, sep=";")
#Modification des colonnes dates

#creation d'une base de données SQL pour stocker les données
con = sql.connect('my.db')
#Mettre le tableau dans SQlite
ventes.to_sql(name='ventes', con=con, if_exists='replace')
produit.to_sql(name='produits', con=con, if_exists='replace')
Employés.to_sql(name='employés', con=con, if_exists='replace')
clients.to_sql(name='clients', con=con, if_exists='replace')
con = sql.connect("my.db")


In [407]:
#from datetime import datetime as dt
#ventes['engage_date']=pd.to_datetime(ventes['engage_date']).dt.strftime("%m/%d/%Y")
#ventes['close_date']=pd.to_datetime(ventes['close_date']).dt.strftime("%m/%d/%Y")

### Description et aperçu des 4 fichiers importés:
**-Table_ventes** : les ID des opportunités, le nom de l'employé , le produit, l'entreprise qui achète, le statut, les dates de début/fin de l'opportunité, le revenue généré

**-Table_produits**: le nom du produit, la série, et le prix suggéré

**-Table_Employés**: le nom de l'employé, le nom du manager, la région

**-Table_clients**: le nom de l'entreprise, secteur, l'année de création, revenu annuel, nombre d'employés, localisation

**Aperçu des 4 fichiers**

In [3]:
print("Aperçu Table_ventes")
ventes = pd.read_sql_query("SELECT * from ventes", con)
display(ventes.head())
print("Aperçu Table_produits")
produits = pd.read_sql_query("SELECT * from produits", con)
display(produits.head())
print("Aperçu Table_Employés")
Employés = pd.read_sql_query("SELECT * from employés", con)
display(Employés.head())
print("Aperçu Table_clients")
clients = pd.read_sql_query("SELECT * from clients", con)
display(clients.head())

Aperçu Table_ventes


,index,opportunity_id,sales_agent,product,account,deal_stage,engage_date,close_date,close_value
0,0,1C1I7A6R,Moses Frase,GTX Plus Basic,Cancity,Won,2016-10-20,2017-03-01,1054.0
1,1,Z063OYW0,Darcel Schlecht,GTX Pro,Isdom,Won,2016-10-25,2017-03-11,4514.0
2,2,EC4QE1BX,Darcel Schlecht,MG Special,Cancity,Won,2016-10-25,2017-03-07,50.0
3,3,MV1LWRNH,Moses Frase,GTX Basic,Codehow,Won,2016-10-25,2017-03-09,588.0
4,4,PE84CX4O,Zane Levy,GTX Basic,Hatfan,Won,2016-10-25,2017-03-02,517.0


Aperçu Table_produits


,index,product,series,sales_price
0,0,GTX Basic,GTX,550
1,1,GTX Pro,GTX,4821
2,2,MG Special,MG,55
3,3,MG Advanced,MG,3393
4,4,GTX Plus Pro,GTX,5482


Aperçu Table_Employés


,index,sales_agent,manager,regional_office
0,0,Anna Snelling,Dustin Brinkmann,Central
1,1,Cecily Lampkin,Dustin Brinkmann,Central
2,2,Versie Hillebrand,Dustin Brinkmann,Central
3,3,Lajuana Vencill,Dustin Brinkmann,Central
4,4,Moses Frase,Dustin Brinkmann,Central


Aperçu Table_clients


,index,account,sector,year_established,revenue,employees,office_location,subsidiary_of
0,0,Acme Corporation,technolgy,1996,1100.04,2822,United States,None
1,1,Betasoloin,medical,1999,251.41,495,United States,None
2,2,Betatech,medical,1986,647.18,1185,Kenya,None
3,3,Bioholding,medical,2012,587.34,1356,Philipines,None
4,4,Bioplex,medical,1991,326.82,1016,United States,None



### Requêtes SQL
*Toutes les requêtes débutent par 'Select' (en rouge) dans le code

### Axe I: Analyse des opportunités créées**

**Requête 1: Selection des opportunités ayant le statut "engagés ou gagnés**

In [4]:
requete1 = pd.read_sql_query("SELECT *from ventes where deal_stage = 'Won'or deal_stage = 'Engaging'", con)
requete1

,index,opportunity_id,sales_agent,product,account,deal_stage,engage_date,close_date,close_value
0,0,1C1I7A6R,Moses Frase,GTX Plus Basic,Cancity,Won,2016-10-20,2017-03-01,1054.0
1,1,Z063OYW0,Darcel Schlecht,GTX Pro,Isdom,Won,2016-10-25,2017-03-11,4514.0
2,2,EC4QE1BX,Darcel Schlecht,MG Special,Cancity,Won,2016-10-25,2017-03-07,50.0
3,3,MV1LWRNH,Moses Frase,GTX Basic,Codehow,Won,2016-10-25,2017-03-09,588.0
4,4,PE84CX4O,Zane Levy,GTX Basic,Hatfan,Won,2016-10-25,2017-03-02,517.0
...,...,...,...,...,...,...,...,...,...
5822,8295,2EBLR9N8,Lajuana Vencill,GTX Basic,Conecom,Won,2017-12-24,2017-12-26,622.0
5823,8296,VDGA4KXA,Violet Mclelland,GTX Plus Basic,Bluth Company,Won,2017-12-24,2017-12-30,1093.0
5824,8297,6WCNNK5J,Maureen Marcano,GTX Pro,Hottechi,Won,2017-12-26,2017-12-29,4433.0
5825,8298,YJTQSZ9D,Gladys Colclough,GTX Plus Basic,Inity,Won,2017-12-27,2017-12-30,1052.0


**Requête 2: Afficher les opportunités les plus anciennes antérieures à juin 2016 et qui qui sont toujours encours(avec un account renseigné)**

In [27]:
requete2 = pd.read_sql_query("SELECT *from ventes where engage_date < '2016-12-31' AND deal_stage = 'Engaging'AND account is not null", con)
display(requete2.head(10))
display(len(requete2))

,index,opportunity_id,sales_agent,product,account,deal_stage,engage_date,close_date,close_value
0,25,UP409DSB,Maureen Marcano,MG Advanced,Ganjaflex,Engaging,2016-11-10,None,None
1,64,0DRC1U9Q,Maureen Marcano,GTX Basic,Green-Plus,Engaging,2016-11-20,None,None
2,79,M7I5O9YU,Corliss Cosme,GTX Basic,Cheers,Engaging,2016-11-23,None,None
3,85,VDIU10RV,Markita Hansen,MG Special,Lexiqvolax,Engaging,2016-11-23,None,None
4,89,579LZ3F9,Daniell Hammack,GTX Plus Basic,J-Texon,Engaging,2016-11-24,None,None
5,90,SU8JNMP4,Kami Bicknell,GTX Plus Basic,Bubba Gump,Engaging,2016-11-24,None,None
6,107,PFFHM7HR,Markita Hansen,MG Advanced,Opentech,Engaging,2016-11-27,None,None
7,172,ICMXZOVL,Daniell Hammack,GTX Pro,Stanredtax,Engaging,2016-12-08,None,None
8,189,MNVMY83P,Cassey Cress,GTX Pro,Plussunin,Engaging,2016-12-10,None,None
9,217,J53D8EOL,Kary Hendrixson,GTX Plus Basic,Hottechi,Engaging,2016-12-14,None,None


19

**Requête 3: Est ce que ce sont les opportunités qui impliquent des produits plus chers vs les opportunités plus récentes?**

In [37]:
requete3 = pd.read_sql_query("SELECT AVG(sales_price) as Prix_Moyen_anciennes_opportunités from ventes join produits using(product) where engage_date <= '2016-12-31' AND deal_stage = 'Engaging'AND account is not null", con)
display(requete3)

requete3bis = pd.read_sql_query("SELECT AVG(sales_price) as Prix_Moyen_opportunités_recentes from ventes join produits using(product) where engage_date > '2016-12-31' AND deal_stage = 'Engaging'AND account is not null", con)
display(requete3bis)

#Avant fin de 2016, les opportunités qui n'ont pas abouti avaient un prix moyen plus elevé

,Prix_Moyen_anciennes_opportunités
0,3600.368421


,Prix_Moyen_opportunités_recentes
0,2492.591286


**Requête 4: Selection pour chaque produit du % d'opportunités perdues par rapport au total des opportunités de ce produit**

In [28]:
requete4 = pd.read_sql_query("SELECT product, round(count(case when deal_stage = 'Lost' then 1 else null END))/count(*)*100 as Pourcentage_Négo_Perdues from ventes group by product ORDER BY Pourcentage_Négo_Perdues DESC ", con)
requete4
#clé de lecture:  Sur le produit MG Advanced => 30% des opportunités crées n'aboutissent pas

,product,Pourcentage_Négo_Perdues
0,MG Advanced,30.453258
1,GTX Plus Basic,28.778019
2,GTX Pro,28.243243
3,GTX Basic,27.920686
4,GTX Plus Pro,27.479339
5,MG Special,26.044821
6,GTK 500,25.000000


In [48]:
#Détail calcul de la requête 4
#lignes d'opportunités perdus
requete_a = pd.read_sql_query("SELECT product, count(product) as Opportunités_perdus from ventes where deal_stage='Lost' group by product", con)
display(requete_a)

#lignes d'oportunité total par produit 
requete_b = pd.read_sql_query("SELECT product, count(product) as Total_opportunités from ventes group by product", con)
display(requete_b)

,product,Opportunités_perdus
0,GTK 500,10
1,GTX Basic,521
2,GTX Plus Basic,398
3,GTX Plus Pro,266
4,GTX Pro,418
5,MG Advanced,430
6,MG Special,430


,product,Total_opportunités
0,GTK 500,40
1,GTX Basic,1866
2,GTX Plus Basic,1383
3,GTX Plus Pro,968
4,GTX Pro,1480
5,MG Advanced,1412
6,MG Special,1651


### Axe II: Analyse Produit, Prix et leur évolution dans le temps

**Requête 5: Selection des produits de la séries GTX/MG du plus cher au moins cher (ici séries avec 3 caractères)**

In [39]:
requete5 = pd.read_sql_query("SELECT * from produits where series like '___' ORDER by sales_price DESC ", con)
display(requete5)
# Même requête pour la série MG
requete5bis = pd.read_sql_query("SELECT * from produits where series like '__' ORDER by sales_price DESC ", con)
display(requete5bis)
#la série GTK 500

,index,product,series,sales_price
0,6,GTK 500,GTK,26768
1,4,GTX Plus Pro,GTX,5482
2,1,GTX Pro,GTX,4821
3,5,GTX Plus Basic,GTX,1096
4,0,GTX Basic,GTX,550


,index,product,series,sales_price
0,3,MG Advanced,MG,3393
1,2,MG Special,MG,55


**Requête 6: Classement automatique des produits en fonction du (fonction de Fenetrage)**

In [61]:
requete6 = pd.read_sql_query("select product, sum(close_value) as Total_Ventes,DENSE_RANK()OVER(ORDER BY sum(close_value) desc)  as classement from ventes group by product",con)
requete6*
#Mais il vient en 6ième position en terme de CA généré

,product,Total_Ventes,classement
0,GTX Pro,3510578.0,1
1,GTX Plus Pro,2629651.0,2
2,MG Advanced,2216387.0,3
3,GTX Plus Basic,705275.0,4
4,GTX Basic,499263.0,5
5,GTK 500,400612.0,6
6,MG Special,43768.0,7


**Requête 7: Les produits dont on a le plus baissé le prix pour assurer la vente c-a-d ceux qui ont le plus de différence entre le prix recommandé et le prix de vente**

In [64]:
requete7 = pd.read_sql_query("SELECT product,deal_stage, sales_price, close_value, close_value-sales_price  as Ecart_Prix, (close_value-sales_price)/sales_price*100  as Evolution_Prix from ventes left join produits using(product) where deal_stage = 'Won'AND close_date is not null group by product " , con)
requete7
# Les produits avec une grande différence de prix sont ceux des séries GTK et GTX mais c'est le MG Spécial qui enregistre le % le plus élevé

,product,deal_stage,sales_price,close_value,Ecart_Prix,Evolution_Prix
0,GTK 500,Won,26768,25897.0,-871.0,-3.253885
1,GTX Basic,Won,550,588.0,38.0,6.909091
2,GTX Plus Basic,Won,1096,1054.0,-42.0,-3.832117
3,GTX Plus Pro,Won,5482,5169.0,-313.0,-5.709595
4,GTX Pro,Won,4821,4514.0,-307.0,-6.367973
5,MG Advanced,Won,3393,3393.0,0.0,0.000000
6,MG Special,Won,55,50.0,-5.0,-9.090909


**Requête 8: "Les produits les plus chers ont-ils une durée de négociation plus longue?**

In [414]:
requete8 = pd.read_sql_query("SELECT AVG(julianday(close_date)-julianday(engage_date)) as Nbre_jours_avant_signature, product from ventes where deal_stage='Won' AND close_date IS NOT NULL group by product ORDER BY Nbre_jours_avant_signature DESC", con)
requete8
#sans surprise, le produit GTK 500 a la date la plus longue de négociation

,Nbre_jours_avant_signature,product
0,64.066667,GTK 500
1,54.765027,GTX Basic
2,51.779817,MG Advanced
3,51.756508,GTX Plus Basic
4,51.726514,GTX Plus Pro
5,51.340479,MG Special
6,48.326475,GTX Pro


**Requête 9: Somme des ventes par année (engage_date) / par mois (close_date) et ensuite pour chaque produit **

Utilisation de engage_date pour avoir 2 années dans l'exemple

In [72]:
#Somme des ventes par an
requete9 = pd.read_sql_query("SELECT strftime('%Y', engage_date) as YEAR, sum(close_value) as Ventes from ventes where deal_stage is not 'Prospecting'  group by YEAR", con)
display(requete9)

#Somme des ventes par mois
requete9bis = pd.read_sql_query("SELECT strftime('%m', close_date) as Month, sum(close_value) as Total_Ventes from ventes where deal_stage = 'Won'AND close_date is not null  group by Month ORDER BY Total_Ventes DESC ", con)
display(requete9bis)
#Les ventes les plus élevés sont réalisées au mois de juin, septembre et mars

#somme des ventes par an par produit
requete9bis2 = pd.read_sql_query("SELECT strftime('%Y', engage_date) as YEAR, product, sum(close_value) as Ventes from ventes where deal_stage is not 'Prospecting'  group by YEAR, product ORDER BY YEAR DESC", con)
display(requete9bis2)
#Le classement en fonction du montant des ventes par produit ne change pas d'une année à l'autre

,YEAR,Ventes
0,2016,523531.0
1,2017,9482003.0


,Month,Total_Ventes
0,06,1338466.0
1,09,1235264.0
2,03,1134672.0
3,12,1131573.0
4,08,1050059.0
5,05,1025713.0
6,11,938943.0
7,10,731980.0
8,04,721932.0
9,07,696932.0


,YEAR,product,Ventes
0,2017,GTK 500,374715.0
1,2017,GTX Basic,466071.0
2,2017,GTX Plus Basic,663308.0
3,2017,GTX Plus Pro,2487088.0
4,2017,GTX Pro,3348189.0
5,2017,MG Advanced,2101915.0
6,2017,MG Special,40717.0
7,2016,GTK 500,25897.0
8,2016,GTX Basic,33192.0
9,2016,GTX Plus Basic,41967.0


**Requête 10: Calculer le taux d’évolution des ventes par année et par mois**

In [70]:
requete10 = pd.read_sql_query("select strftime('%Y', engage_date) as Année, sum(close_value) as CA,(sum(close_value)-lag(sum(close_value), 1, 0) over(order by strftime('%Y', engage_date)))/ lag(sum(close_value), 1, 0) over(order by strftime('%Y', engage_date))  as taux_evol from ventes where deal_stage='Won'  group by Année", con)
display(requete10)

requete10bis = pd.read_sql_query("select strftime('%m', engage_date) as mois, sum(close_value) as CA,(sum(close_value)-lag(sum(close_value), 1, 0) over(order by strftime('%m', engage_date)))/ lag(sum(close_value), 1, 0) over(order by strftime('%m', engage_date))  as taux_evol from ventes where deal_stage='Won'  group by Mois", con)
display(requete10bis)

,Année,CA,taux_evol
0,2016,523531.0,NaN
1,2017,9482003.0,17.111636


,mois,CA,taux_evol
0,01,408451.0,NaN
1,02,757978.0,0.855738
2,03,1004017.0,0.324599
3,04,1035338.0,0.031196
4,05,852638.0,-0.176464
5,06,1080280.0,0.266986
6,07,1049172.0,-0.028796
7,08,706513.0,-0.326599
8,09,1037048.0,0.467840
9,10,1014977.0,-0.021283


### Axe III: Analyse Entreprise BtoB

**Requête 11: Top 10 des entreprises qui ont en moyenne de jour de négociation plus longue"(la durée entre la date d'engagement des discussions et la date de ventes) + les informations sur l'entreprise**

In [74]:
requete11 = pd.read_sql_query("SELECT AVG(julianday(close_date)-julianday(engage_date)) as Nbre_jours_avant_signature, account,sector,year_established,revenue,employees,office_location,subsidiary_of from ventes left join clients using(account) where deal_stage='Won' AND close_date IS NOT NULL group by account ORDER BY Nbre_jours_avant_signature DESC LIMIT 10", con)
requete11
#Dans ce top, nous avons plus de difficulté avec les entreprises du secteur médical en 1ier (3 entreprises/10)

,Nbre_jours_avant_signature,account,sector,year_established,revenue,employees,office_location,subsidiary_of
0,73.969697,Zoomit,entertainment,1992,324.19,978,United States,None
1,67.828571,Silis,medical,1994,2818.38,6290,United States,None
2,64.166667,Codehow,software,1998,2714.90,2641,United States,Acme Corporation
3,64.055556,Finhigh,finance,2006,1102.43,1759,United States,None
4,62.705882,Zathunicon,retail,2010,71.12,144,United States,None
5,61.818182,Golddex,finance,2008,52.50,165,United States,None
6,61.434783,Labdrill,medical,1985,2741.37,9226,United States,None
7,59.693333,Rangreen,technolgy,1987,2938.67,8775,Panama,None
8,59.507937,Streethex,retail,1988,1376.80,1165,Belgium,None
9,58.276190,Condax,medical,2017,4.54,9,United States,None


**Requête 12: Classidfication des clients par catégorie (Grands_comptes/Comptes_moyens/Petits_comptes) en fonction du montant de ventes + les informations sur le secteur de l'entreprise**

In [75]:
requete12 = pd.read_sql_query("select account, sector, revenue, count(product) as Nbre_Produit_acheté, case when close_value <1000 then 'Petits_comptes' when close_value BETWEEN 1000 AND 5000 then 'Comptes_moyens' else 'Grands_comptes' end as Catégories_Entreprise from ventes left join clients using(account) group by account having account is not null" , con)
requete12

,account,sector,revenue,Nbre_Produit_acheté,Catégories_Entreprise
0,Acme Corporation,technolgy,1100.04,68,Petits_comptes
1,Betasoloin,medical,251.41,68,Grands_comptes
2,Betatech,medical,647.18,92,Petits_comptes
3,Bioholding,medical,587.34,94,Petits_comptes
4,Bioplex,medical,326.82,53,Petits_comptes
...,...,...,...,...,...
80,Zencorporation,technolgy,40.79,58,Petits_comptes
81,Zoomit,entertainment,324.19,58,Petits_comptes
82,Zotware,software,4478.47,75,Petits_comptes
83,Zumgoity,medical,441.08,63,Comptes_moyens


**Requête 13: Les 10 employés qui ont créé le plus d'opportunités/ le montant total de leur vente/ le nombre de produits qu'ils ont vendus ainsi que le nom de leur manager et la région**

In [76]:
requete13 = pd.read_sql_query("SELECT count(opportunity_id) as Nbre_opportunites, sum(close_value) as Montant_ventes, count(DISTINCT product) as Nbre_Produits_vendus, sales_agent, manager, regional_office from ventes left join employés using(sales_agent)  group by sales_agent ORDER BY Nbre_opportunites DESC LIMIT 10", con)
requete13

,Nbre_opportunites,Montant_ventes,Nbre_Produits_vendus,sales_agent,manager,regional_office
0,747,1153214.0,7,Darcel Schlecht,Melvin Marxen,Central
1,451,478396.0,6,Vicki Laflamme,Celia Rouche,West
2,448,275056.0,5,Anna Snelling,Dustin Brinkmann,Central
3,438,454298.0,6,Kary Hendrixson,Summer Sewald,West
4,362,316456.0,6,Kami Bicknell,Summer Sewald,West
5,361,187693.0,5,Versie Hillebrand,Dustin Brinkmann,Central
6,349,430068.0,6,Zane Levy,Summer Sewald,West
7,346,450489.0,6,Cassey Cress,Rocco Neubert,East
8,345,284886.0,6,Jonathan Berthelot,Melvin Marxen,Central
9,317,345674.0,6,Gladys Colclough,Melvin Marxen,Central


**Requête 14: Le top 3 des meilleurs employés en termes de chiffre d'affaire par mois pour chaque année**

In [98]:
requete13 = pd.read_sql_query("select Mois, sales_agent, CA_employés,  classement_mois , manager, regional_office from (select strftime('%m', close_date) as Mois, sales_agent, sum(close_value) as CA_employés, dense_rank() over(partition by strftime('%m', close_date) order by sum(close_value) desc) as classement_mois from ventes  where deal_stage='Won' group by sales_agent, Mois) as temp join employés using(sales_agent) where classement_mois between 1 and 3", con)
requete13

,Mois,sales_agent,CA_employés,classement_mois,manager,regional_office
0,07,Marty Freudenburg,40506.0,2,Melvin Marxen,Central
1,10,Marty Freudenburg,45073.0,3,Melvin Marxen,Central
2,07,Gladys Colclough,40217.0,3,Melvin Marxen,Central
3,03,Darcel Schlecht,112255.0,1,Melvin Marxen,Central
4,04,Darcel Schlecht,92830.0,1,Melvin Marxen,Central
5,05,Darcel Schlecht,95118.0,1,Melvin Marxen,Central
6,06,Darcel Schlecht,122127.0,1,Melvin Marxen,Central
7,07,Darcel Schlecht,96411.0,1,Melvin Marxen,Central
8,08,Darcel Schlecht,140273.0,1,Melvin Marxen,Central
9,09,Darcel Schlecht,136534.0,1,Melvin Marxen,Central
